## Набор данных Футболисты

В качестве задания вам предлагается повторить данные действия для набора данных
- [Blood Transfusion Service Center Data Set](http://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center)

In [ ]:
import pandas as pd

Набор данных о футболистах.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/yakushinav/omo/main/data/data_football_profile.csv', sep='\t')

In [ ]:
df.columns

Функция возвращает DataFrame (то есть таблицу), однако затем приобретает ещё много важных параметров, среди которых:

* sep — разделитель данных, по умолчанию ',';
* decimal — разделитель числа на целую и дробную часть, по умолчанию'.';
* names — список с названиями колонок, не обязательный параметр;
* skiprows — если файл содержит системную информацию, можно просто её пропустить. Необязательный параметр.

С помощью функции ```head``` можем посмотреть на первые несколько строк нашего датасета:

In [ ]:
df.head()

Удалим колонку Value, к которой мы не знаем точную интерпретацию:

In [ ]:
df.drop(['Value'], axis=1, inplace=True)

In [ ]:
# последние несколько строк:
df.tail(3)

Посмотрим на размер нашего датасета. Первое число – количество строк (наблюдений), второе – количество столбцов (признаков):

In [ ]:
df.shape

Если вы хотите переименовать какую-то переменную, воспользуйтесь ```rename```:

In [ ]:
df.rename({'Wage' : 'Salary'}, axis='columns', inplace=True)

In [ ]:
df.columns

Давайте посмотрим на информацию о датасете. В .info() можно передать дополнительные параметры, среди которых:

* verbose: печатать ли информацию о DataFrame полностью (если таблица очень большая, то некоторая информация может потеряться);
* memory_usage: печатать ли потребление памяти (по умолчанию используется True, но можно поставить либо False, что уберёт потребление памяти, либо 'deep' , что подсчитает потребление памяти более точно);
* null_counts: подсчитывать ли количество пустых элементов (по умолчанию True).

In [ ]:
df.info()

Можно вывести только тип данных в каждой колонке:

In [ ]:
df.dtypes

Заметим, что зарплата у нас записана строкой.

In [ ]:
def fix_wage(x):
    # отрезаем $
    x = x[:-1]
    # меняем запятую на точку
    x = x.replace(',', '.')
    return float(x)

In [ ]:
df['Salary'] = df['Salary'].apply(fix_wage)

Метод describe показывает основные статистические характеристики данных по каждому числовому признаку (типы int64 и float64): число непропущенных значений, среднее, стандартное отклонение, диапазон, медиану, 0.25 и 0.75 квартили.

In [ ]:
df.describe()

Чтобы посмотреть статистику по нечисловым признакам (например, по строчным (object) или булевым (bool) данным), нужно явно указать интересующие нас типы в параметре метода describe include:

In [ ]:
df.describe(include = ['object'])

Было бы полезно узнать, много ли у нас пропусков в датасете.

In [ ]:
df.isna().sum()
print(df.dtypes)

Чтобы удалить пропуски из данных, нужно вопспользоваться ```df.dropna()```, либо заполнить их значениями (например, средним) -  ```df.fillna(df['column_name'].mean())``` .
Если в датасете содержатся дубликаты строк - воспользуйтесь методом ```df.drop_duplicates()```.

In [ ]:
# заполним количественные переменные средними значениями (медианой)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Salary'] = df['Salary'].fillna(df['Salary'].median())

# у оставшихся переменных удалим строки с пропусками
df.dropna(inplace=True)

df.isna().sum()

Выведем уникальные значения по возрасту и сколько раз каждое из них встречается в датасете (по убыванию). 

In [ ]:
df['Age'].value_counts()

Чтобы вывести уникальные значения в столбце или их количество, нужно использовать ```unique``` и ```nunique``` соответственно. Посмотрим, сколько у нас уникальных футбольных клубов. 

In [ ]:
print('Всего {} футбольных клубов'.format(df['Club'].nunique()))

In [ ]:
df['Club'].unique()[:10]

Посмотрим на среднюю зарплату по клубу:

In [ ]:
grouped = df.groupby('Club', as_index=False)['Salary'].mean()
# добавим сортировку по убыванию
grouped.sort_values(by='Salary', ascending=False)

Добавим еще подсчет минимума, максимума и медианы по каждой группе:

In [ ]:
df.groupby('Club')['Salary'].agg(['mean', 'min', 'max', 'median'])

Сгруппируем одновременно по стране и клубу:

In [ ]:
df.groupby(['Nationality', 'Club'], as_index=False)['Salary'].mean()

Добавим сортировку внутри групп:

In [ ]:
df.groupby(['Nationality', 'Club']).apply(lambda x: x.sort_values(by='Salary', ascending=False))

Теперь удалим лишние колонки. Обратите внимание на обратный слэш, это line continuation character.

In [ ]:
df.groupby(['Nationality', 'Club']).apply(lambda x: x.sort_values(by='Salary', ascending=False)).\
                                                    drop(['Nationality', 'Club'], axis=1)

Посчитаем арифметическое среднее, моду и медиану возраста футболистов (количественной переменной):

In [ ]:
print('Среднее:', round(df['Age'].mean(), 2), 
      'Медиана:', df['Age'].median(), 
      'Мода:', df['Age'].mode()[0])

Для качественных переменных с помощью pandas можно вывести моду. Посмотрим на самую часто встречающуюся национальность:

In [ ]:
df['Nationality'].mode()

Часто возникает необходимость выбрать данные из DataFrame по определённому условию. Например, если в уже известном нам наборе данных о футболистах мы хотим выбрать только тех, у кого возраст больше 20 лет, используется следующий код:

In [ ]:
df[df.Age > 20]

Выберем футболистов, возраст которых больше среднего возраста футболистов, при условии, что они принадлежат ФК Барселона (Club == 'FC Barcelona').

In [ ]:
df[(df.Age > df.Age.mean()) & (df.Club == 'FC Barcelona')]

Чтобы объединить данные из нескольких датасетов по ключу (общей колонке), в pandas можно воспользовать встроенными аналогами SQL методов. В метод ```join``` в качестве аргумента how нужно указать тип объединения датасетов: inner, outer, left или right. 

In [ ]:
df_info = pd.read_csv('https://raw.githubusercontent.com/yakushinav/omo/main/data/data_football_info.csv', sep='\t')

joined_dfs = df_info.set_index('Name').join(df.set_index('Name'), how='inner').reset_index()
joined_dfs.head(5)

В качестве задания вам предлагается повторить данные действия по визуализации и классификации для набора данных
- [Blood Transfusion Service Center Data Set](http://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center)